In [1]:
# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------

import argparse
import torch, os, sys
#from deepspeed.ops.adam import FusedAdam
from azureml.core import Run
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, RichModelSummary
from os.path import join
from datasets.mae_datasets import make_dataloaders
import argparse
from utils.utils import display_environment
from utils.openmpi import set_strategy
from torch import optim
from models.models_mae import mae_vit_huge_patch14_dec512d8b
import timm
from torch import optim, nn, utils
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch, os, sys
#from deepspeed.ops.adam import FusedAdam
from azureml.core import Run
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, RichModelSummary
from os.path import join
from datasets.mae_datasets import make_dataloaders
import argparse
from utils.utils import display_environment
from utils.openmpi import set_strategy, OpenMPIClusterEnvironment
from torch import optim
from models.models_mae import mae_vit_huge_patch14_dec512d8b
import timm

Extension horovod.torch has not been built: /anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/horovod/torch/mpi_lib_v2.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


In [2]:
class MAELightning(LightningModule):
    def __init__(self, args):
        super().__init__()
        self.args = args       
        self.model = mae_vit_huge_patch14_dec512d8b()

    def training_step(self, batch, _):
        # Get the images and labels.
        X = batch['image']

        # Compute the training loss.
        loss, _, _ = self.model(X)

        # Log the training loss.
        self.log("train_loss", loss, prog_bar=True, sync_dist=True)

        return loss

    def validation_step(self, batch, _):
        # Get the images and labels.
        X = batch['image']
        val_loss, _, _ = self.model(X)
       
        self.log("val_loss", val_loss.item(), prog_bar=True, sync_dist=True) 
        
        return {"val_loss":val_loss.item()}

    def configure_optimizers(self):
        # Make the optimizer and learning rate scheduler.
        optimizer = optim.AdamW(
            self.parameters(), lr=self.args.learning_rate, weight_decay=self.args.weight_decay
        )
        return [optimizer]

In [3]:
class Args:
    dataset_directory_name="azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_185853_UTC/ImageNet_1000_sm/imagenet-mini/"
    trainset="azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190730_UTC/ImageNet_1000_train.csv"
    valset="azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/antonslutsky-rg/workspaces/gpu-workspace/datastores/workspaceblobstore/paths/UI/2023-12-05_190801_UTC/ImageNet_1000_val.csv" 
    num_epochs=500
    batch_size=10
    num_workers=1
    num_nodest=1
    num_devices=1
    img_size=224
    strategy="deepspeed_stage_1"
    experiment_name="minizilla_01"
    precision=32
    learning_rate=1e-4
    weight_decay=0
    num_nodes=1

args = Args()
args

In [4]:
%env PL_GLOBAL_SEED = 102938
%env PL_SEED_WORKERS = 1
%env AZ_BATCH_MASTER_NODE = 127.0.0.1:6000
%env AZ_BATCHAI_MPI_MASTER_NODE = 127.0.0.1
%env MASTER_ADDR = 127.0.0.1
%env MASTER_ADDRESS = 
%env MASTER_PORT = 6105
%env RANK = 0
%env NODE_RANK = 0
%env LOCAL_RANK = 0
%env GLOBAL_RANK = 
%env WORLD_SIZE = 16
%env NCCL_SOCKET_IFNAME = eth0
%env OMPI_COMM_WORLD_RANK = 0
%env OMPI_COMM_WORLD_LOCAL_RANK = 0
%env OMPI_COMM_WORLD_SIZE = 1
%env OMPI_COMM_WORLD_LOCAL_SIZE = 1

env: PL_GLOBAL_SEED=102938
env: PL_SEED_WORKERS=1
env: AZ_BATCH_MASTER_NODE=127.0.0.1:6000
env: AZ_BATCHAI_MPI_MASTER_NODE=127.0.0.1
env: MASTER_ADDR=127.0.0.1
env: MASTER_ADDRESS=
env: MASTER_PORT=6105
env: RANK=0
env: NODE_RANK=0
env: LOCAL_RANK=0
env: GLOBAL_RANK=
env: WORLD_SIZE=16
env: NCCL_SOCKET_IFNAME=eth0
env: OMPI_COMM_WORLD_RANK=0
env: OMPI_COMM_WORLD_LOCAL_RANK=0
env: OMPI_COMM_WORLD_SIZE=1
env: OMPI_COMM_WORLD_LOCAL_SIZE=1


In [4]:

from pytorch_lightning.strategies import DeepSpeedStrategy

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
# os.environ["OMPI_COMM_WORLD_LOCAL_RANK"] = "0"
# os.environ["OMPI_COMM_WORLD_RANK"] = "0"
# os.environ["AZ_BATCHAI_MPI_MASTER_NODE"] = "127.0.0.1"


# If running on azure, get the active tracking uri and run id
# otherwise, use the workspace to get a tracking uri
active_run = Run.get_context() #  active run azureml object
# offline = False
try:
    print(active_run.experiment)
    tracking_uri=active_run.experiment.workspace.get_mlflow_tracking_uri()
    run_id = active_run.id
except:
    print(f"WARNING ! Could not connect to the MLFlow tracking uri, please check !") #offline = True

seed_everything(102938, workers = True)

print("creating training and validation sets...")
train_loader , val_loader = make_dataloaders(args)
print("done")


print("creating model...")
model = MAELightning(args=args)
print("done")

# logger = MLFlowLogger(
#             experiment_name=args.experiment_name,
#             tracking_uri=tracking_uri,
#             run_id=run_id
# )

display_environment("__main__")

checkpoint_callback = ModelCheckpoint(monitor="val_loss", 
                                        mode="min",
                                        save_top_k=999,
                                        verbose=True,
                                        dirpath="./outputs/checkpoints/",
                                        filename="{epoch}-{val_loss:.2f}",
                                        save_weights_only=True,
                                        auto_insert_metric_name=True)  

lr_monitor = LearningRateMonitor(logging_interval='step')

model_summary = RichModelSummary(max_depth=1)

trainer = Trainer(
    num_nodes=args.num_nodes,
    accelerator='gpu',
    devices=args.num_devices,
    log_every_n_steps=1,
    logger=None,
    num_sanity_val_steps=2,
    max_epochs=args.num_epochs,
    enable_model_summary=False,
    callbacks = [checkpoint_callback,lr_monitor,model_summary],
    # strategy=DeepSpeedStrategy(
    #         stage = 1,
    #         cluster_environment = OpenMPIClusterEnvironment(devices=args.num_devices)
    #     ),
    strategy = "deepspeed_stage_1",
    precision=args.precision
)

display_environment("__main__")

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

print(f"""trainer.local_rank: {trainer.local_rank}
trainer.global_rank : {trainer.global_rank}
trainer.world_size : {trainer.world_size}
""")


Global seed set to 102938


WARNING ! Could not connect to the MLFlow tracking uri, please check !
creating training and validation sets...
Creating dataloaders.......... !!!!!!!!!!!!!!!!!!!!!


ValueError: Protocol not known: azureml

In [6]:
print("Cuda support:", torch.cuda.is_available(),":", torch.cuda.device_count(), "devices")

Cuda support: True : 1 devices


In [8]:
!pip show cudatoolkit